In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import geopandas as gpd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [9]:
suburb_gpd = gpd.read_file(r"../Shapefiles/Suburbs_2021")
suburb_master = pd.read_excel(r"../suburbs_compressed.xlsx", sheet_name="Crime Dataset")
SAL_code = pd.read_csv(r"../SubSAL_code.csv")

In [10]:
suburb_feat = suburb_master[["Assault (Family)_2015-16_August", "Suburbs"]]
suburb_feat_SAL = pd.merge(left=suburb_feat, right=SAL_code,how="left", left_on="Suburbs", right_on="0")

In [11]:
suburb_feat_SAL = suburb_feat_SAL[["Assault (Family)_2015-16_August", "SAL_CODE"]]
sal_with_feat = pd.merge(left=suburb_gpd, right=suburb_feat_SAL, how="left", left_on="SAL_CODE21", right_on="SAL_CODE")
sal_with_feat.dropna(inplace=True)
sal_with_feat["Radius"] = (sal_with_feat["SHAPE_Area"]/3.14)**0.5

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

# Our 2-dimensional distribution will be over variables X and Y
N = 40
X = np.linspace(-5, 5, N)
Y = np.linspace(-5, 5, N)
X, Y = np.meshgrid(X, Y)

# Mean vector and covariance matrix
mu = np.array([0., 0.])
Sigma = np.array([[ 1. , -0.5], [-0.5,  1.]])

# Pack X and Y into a single 3-dimensional array
pos = np.empty(X.shape + (2,))
pos[:, :, 0] = X
pos[:, :, 1] = Y

def multivariate_gaussian(pos, mu, Sigma):
    """Return the multivariate Gaussian distribution on array pos."""

    n = mu.shape[0]
    Sigma_det = np.linalg.det(Sigma)
    Sigma_inv = np.linalg.inv(Sigma)
    N = np.sqrt((2*np.pi)**n * Sigma_det)
    # This einsum call calculates (x-mu)T.Sigma-1.(x-mu) in a vectorized
    # way across all the input variables.
    fac = np.einsum('...k,kl,...l->...', pos-mu, Sigma_inv, pos-mu)

    return np.exp(-fac / 2) / N

# The distribution on the variables X, Y packed into pos.
Z = multivariate_gaussian(pos, mu, Sigma)

mu2 = np.array([1, 2])
Sigma2 = np.array([[ 2. , 0], [0,  2.]])
pos2 = np.empty(X.shape + (2,))
pos2[:, :, 0] = X
pos2[:, :, 1] = Y
z_2 = multivariate_gaussian(pos2, mu2, Sigma2)


In [6]:
%matplotlib qt
import matplotlib.pyplot as plt
from scipy import stats

xmin = sal_with_feat.centroid.x.min()
xmax = sal_with_feat.centroid.x.max()
ymin = sal_with_feat.centroid.y.min()
ymax = sal_with_feat.centroid.y.max()

X, Y = np.mgrid[xmin:xmax:1000j, ymin:ymax:1000j]
positions = np.vstack([X.ravel(), Y.ravel()])
Z = []

for i in tqdm(range(len(sal_with_feat))):
# for i in range(0, 2):
    # print(i)
    m1, m2,feat, radius = sal_with_feat.centroid.x.iloc[i], sal_with_feat.centroid.y.iloc[i], sal_with_feat["Assault (Family)_2015-16_August"].iloc[i], sal_with_feat.Radius.iloc[i]
    mu = np.array([m1, m2])
    Sigma = np.array([[ 1. , 0], [0,  1.]])*radius*0.05

    # Pack X and Y into a single 3-dimensional array
    pos = np.empty(X.shape + (2,))
    pos[:, :, 0] = X
    pos[:, :, 1] = Y
        
    Z.append( multivariate_gaussian(pos, mu, Sigma))


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""
100%|██████████| 7/7 [00:02<00:00,  3.17it/s]


In [11]:
result = np.sum(Z, axis=0)
##result is the  final heatmap
fig, ax = plt.subplots()
sal_with_feat.plot(ax=ax, color = "None")
ax.imshow(np.rot90(result), cmap=plt.cm.gist_earth_r,
          extent=[xmin, xmax, ymin, ymax])
ax.plot(m1, m2, 'k.', markersize=2)
ax.set_xlim([xmin, xmax])
ax.set_ylim([ymin, ymax])
sal_with_feat.centroid.plot(ax=ax, color="DarkBlue",markersize =1)
plt.show()